<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Movie-recommendation" data-toc-modified-id="Movie-recommendation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Movie recommendation</a></span><ul class="toc-item"><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#Evaluation-Protocol" data-toc-modified-id="Evaluation-Protocol-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Evaluation Protocol</a></span></li><li><span><a href="#Models" data-toc-modified-id="Models-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Models</a></span><ul class="toc-item"><li><span><a href="#ALS" data-toc-modified-id="ALS-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-collaborative-filtering.html#explicit-vs-implicit-feedback" target="_blank">ALS</a></a></span></li><li><span><a href="#Ваша-формулировка" data-toc-modified-id="Ваша-формулировка-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Ваша формулировка</a></span></li></ul></li><li><span><a href="#Evaluation-Results" data-toc-modified-id="Evaluation-Results-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Evaluation Results</a></span></li></ul></li></ul></div>

# Movie recommendation

Ваша задача - рекомендация фильмов для пользователей


In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

## Dataset 

`MovieLens-25M`

In [2]:
DATA_PATH = '/workspace/data/ml-10M100K'

RATINGS_PATH = os.path.join(DATA_PATH, 'ratings.csv')
MOVIES_PATH = os.path.join(DATA_PATH, 'movies.csv')
TAGS_PATH = os.path.join(DATA_PATH, 'tags.csv')

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *


ratings_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + RATINGS_PATH)

## Evaluation Protocol

Так как мы хотим оценивать качество разных алгоритмов рекомендаций, то в первую очередь нам нужно определить
* Как разбить данные на `Train`/`Validation`/`Test`;
* Какие метрики использовать для оценки качества.

In [4]:
from pyspark.sql.window import Window


timestamp_rank_window = Window().orderBy('timestamp')


TRAIN_SHARE = .8
VAL_SHARE = .1
TEST_SHARE = .1


ranked_ratings_df = ratings_df \
    .withColumn('timestamp_rank', F.percent_rank().over(timestamp_rank_window))
train_df = ranked_ratings_df \
    .filter(F.col('timestamp_rank') <= TRAIN_SHARE) \
    .drop('timestamp_rank')
val_df = ranked_ratings_df \
    .filter((TRAIN_SHARE < F.col('timestamp_rank')) & (F.col('timestamp_rank') <= TRAIN_SHARE + VAL_SHARE)) \
    .drop('timestamp_rank')
test_df = ranked_ratings_df \
    .filter(F.col('timestamp_rank') > TRAIN_SHARE + VAL_SHARE) \
    .drop('timestamp_rank')

In [5]:
def get_users(df):
    return df \
        .select('userId') \
        .distinct()


train_users = get_users(train_df)
val_users = get_users(val_df)
test_users = get_users(test_df)

common_users = train_users \
    .join(val_users, on='userId') \
    .join(test_users, on='userId')

train_df = train_df.join(common_users, on='userId')
val_df = val_df.join(common_users, on='userId')
test_df = test_df.join(common_users, on='userId')

In [6]:
print(train_df.count(), val_df.count(), test_df.count())
print(train_users.count(), val_users.count(), test_users.count())
print(common_users.count())

905582 148144 102983
59522 7924 8287
1628


## Models

Теперь мы можем перейти к формулировке задачи в терминах машинного обучения.

Одна из формулировок, к которой мы сведем нашу задачу - **Matrix Completetion**. Данную задачу будем решать с помощью `ALS`

### [ALS](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html#explicit-vs-implicit-feedback)

In [8]:
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=10, regParam=.1, userCol='userId', itemCol='movieId', ratingCol='rating', 
          coldStartStrategy='drop', rank=10, implicitPrefs=False, nonnegative=True)
als_model = als.fit(train_df)

Покажите для выбранных вами фильмов топ-20 наиболее похожих фильмов

In [9]:
movies_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + MOVIES_PATH) \
    .drop('genres')

selected_movies = movies_df \
    .select('movieId') \
    .limit(10)

movie_factors = als_model.itemFactors \
    .withColumnRenamed('id', 'movieId')

selected_movies_factors = selected_movies \
    .join(movie_factors, on='movieId') \
    .withColumnRenamed('movieId', 'selectedId') \
    .withColumnRenamed('features', 'selectedFeatures')

In [10]:
from scipy.spatial.distance import cosine


def cosine_dist(selected_features, features):
    return float(cosine(selected_features, features))

cosine_dist_udf = F.udf(cosine_dist, FloatType())

In [11]:
N_NEIGHBOURS = 20

movie_dists = selected_movies_factors \
    .join(movie_factors, on=F.col('movieId') != F.col('selectedId')) \
    .withColumn('d', cosine_dist_udf('selectedFeatures', 'features')) \
    .drop('selectedFeatures') \
    .drop('features')
    
dist_window = Window().partitionBy('selectedId').orderBy('d')

neighbours = movie_dists \
    .withColumn('rank', F.rank().over(dist_window)) \
    .filter(F.col('rank') <= N_NEIGHBOURS) \
    .withColumnRenamed('movieId', 'neighbourId') \
    .drop('rank')

In [12]:
neighbours \
    .join(movies_df, on=F.col('selectedId') == F.col('movieId')) \
    .drop('movieId') \
    .withColumnRenamed('title', 'selectedTitle') \
    .join(movies_df, on=F.col('neighbourId') == F.col('movieId')) \
    .drop('movieId') \
    .drop('selectedId') \
    .drop('neighbourId') \
    .collect()

[Row(d=0.0020556366071105003, selectedTitle='Toy Story (1995)', title='Toy Story 2 (1999)'),
 Row(d=0.006429308094084263, selectedTitle='Toy Story (1995)', title='Perfect Candidate'),
 Row(d=0.006595601327717304, selectedTitle='Toy Story (1995)', title='Touch (1997)'),
 Row(d=0.006684896536171436, selectedTitle='Toy Story (1995)', title='WarGames (1983)'),
 Row(d=0.006723959464579821, selectedTitle='Toy Story (1995)', title="Bug's Life"),
 Row(d=0.007768779993057251, selectedTitle='Toy Story (1995)', title='Mr. Mom (1983)'),
 Row(d=0.007785863243043423, selectedTitle='Toy Story (1995)', title='My Cousin Vinny (1992)'),
 Row(d=0.008387277834117413, selectedTitle='Toy Story (1995)', title='Foul Play (1978)'),
 Row(d=0.008879751898348331, selectedTitle='Toy Story (1995)', title='Trading Places (1983)'),
 Row(d=0.008921408094465733, selectedTitle='Toy Story (1995)', title='Court Jester'),
 Row(d=0.009236413054168224, selectedTitle='Toy Story (1995)', title='From Russia with Love (1963)'),


### Ваша формулировка

На лекции было еще несколько ML формулировок задачи рекомендаций. Выберете одну из них и реализуйте метод.

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

## Evaluation Results

Сравните реализованные методы с помощью выбранных метрик. Не забывайте про оптимизацию гиперпараметров.

In [13]:
def get_users(df):
    return df \
        .select('userId') \
        .distinct()


def get_relevant_recs(recs, gt):
    return recs.join(gt, on=(recs.userId == gt.userId) & (recs.movieId == gt.movieId)) \
        .drop(gt.movieId) \
        .drop(gt.userId)

In [15]:
def precision_at_k(recs, gt, k):
    relevant_recs = get_relevant_recs(recs, gt) \
        .filter(F.col('rank') <= k)
    all_users = get_users(recs)
    return relevant_recs \
        .groupBy(F.col('userId')) \
        .agg(F.count('movieID').alias('relevant')) \
        .withColumn('prec', F.col('relevant') / k) \
        .drop('relevant') \
        .join(all_users, on='userId', how='right') \
        .fillna(0) \
        .select(F.avg('prec')).collect()

In [ ]:
def mean_average_precision(rec, gt):
    pass

In [16]:
def hr(recs, gt):
    relevant_recs = get_relevant_recs(recs, gt)
    return get_users(relevant_recs).count() / get_users(recs).count()

In [17]:
from pyspark.sql import Row


def flat_recs(recs):
    return ([
        (recs.userId, movieId, i)
        for i, (movieId, _) in enumerate(recs.recommendations, 1)
    ])

def recommend(model, n):
    return model.recommendForAllUsers(20) \
        .rdd \
        .flatMap(flat_recs) \
        .map(lambda r: Row(userId=r[0], movieId=r[1], rank=r[2])) \
        .toDF()

als_recommndations = recommend(als_model, 20)

In [19]:
precision_at_k(als_recommndations, val_df, 10), hr(als_recommndations, val_df)

([Row(avg(prec)=0.007555282555282543)], 0.18181818181818182)

In [20]:
als_recommndations.head(5)

[Row(movieId=8621, rank=1, userId=9900),
 Row(movieId=8188, rank=2, userId=9900),
 Row(movieId=106, rank=3, userId=9900),
 Row(movieId=33124, rank=4, userId=9900),
 Row(movieId=8189, rank=5, userId=9900)]

In [21]:
val_df.head(5)

[Row(userId=51, movieId=45447, rating=4.0, timestamp=1165437501),
 Row(userId=51, movieId=45720, rating=4.0, timestamp=1165437536),
 Row(userId=78, movieId=44694, rating=2.0, timestamp=1179550301),
 Row(userId=78, movieId=111, rating=4.0, timestamp=1176683110),
 Row(userId=78, movieId=34405, rating=4.5, timestamp=1176683162)]

In [22]:
!pip3.5 install hyperopt

In [23]:
from hyperopt import fmin, tpe, hp, Trials

def als_obj(config):
    model = ALS(**config).fit(train_df)
    recommendations = recommend(model, 10)
    return precision_at_k(model, val, 10)

In [24]:
space = {
    'userCol': 'userId', 
    'itemCol': 'movieId', 
    'ratingCol': 'rating', 
    'coldStartStrategy': 'drop', 
    'maxIter': hp.choice('maxIter', [5, 10, 20]), 
    'regParam': hp.choice('regParam', [0., .001, .01, .1, .3, 1, 3]), 
    'rank': hp.choice('rank', [5, 10, 20, 50]), 
    'implicitPrefs': False, 
    'nonnegative': True
}

trials = Trials()
best = fmin(fn=als_obj,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

job exception: An error occurred while calling o437.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 309.0 failed 1 times, most recent failure: Lost task 0.0 in stage 309.0 (TID 2852, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.io.ReadAheadInputStream.<init>(ReadAheadInputStream.java:106)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:81)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:156)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getIterator(UnsafeExternalSorter.java:647)
	at org.apache.spark.sql.execution.ExternalAppendOnlyUnsafeRowArray.generateIterator(ExternalAppendOnlyUnsafeRowArray.scala:171)
	at org.apache.spark.sql.execution

  0%|          | 0/20 [00:46<?, ?trial/s, best loss=?]


Traceback (most recent call last):
  File "/usr/lib64/python3.5/socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.5/socketserver.py", line 341, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.5/socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.5/socketserver.py", line 681, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializers.py", line 717, in read_int
    raise EOFError
EOFError
--- Logging error ---
ERROR:py4j.java_gateway:An error occur

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44283)
Traceback (most recent call last):
  File "/usr/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-470847b67043>", line 18, in <module>
    trials=trials)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 482, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/base.py", line 686, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 509, in fmin
    rval.exhaust()
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 330, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 286, in run
    self.serial_evaluate()
  File "/usr/lib/python3.5/site-packa

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44283)
Traceback (most recent call last):
  File "/usr/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-470847b67043>", line 18, in <module>
    trials=trials)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 482, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/base.py", line 686, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 509, in fmin
    rval.exhaust()
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 330, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 286, in run
    self.serial_evaluate()
  File "/usr/lib/python3.5/site-packa

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44283)
Traceback (most recent call last):
  File "/usr/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-470847b67043>", line 18, in <module>
    trials=trials)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 482, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/base.py", line 686, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 509, in fmin
    rval.exhaust()
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 330, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 286, in run
    self.serial_evaluate()
  File "/usr/lib/python3.5/site-packa

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44283)
Traceback (most recent call last):
  File "/usr/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-470847b67043>", line 18, in <module>
    trials=trials)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 482, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/base.py", line 686, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 509, in fmin
    rval.exhaust()
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 330, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 286, in run
    self.serial_evaluate()
  File "/usr/lib/python3.5/site-packa

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44283)
Traceback (most recent call last):
  File "/usr/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-470847b67043>", line 18, in <module>
    trials=trials)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 482, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/base.py", line 686, in fmin
    show_progressbar=show_progressbar,
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 509, in fmin
    rval.exhaust()
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 330, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "/usr/lib/python3.5/site-packages/hyperopt/fmin.py", line 286, in run
    self.serial_evaluate()
  File "/usr/lib/python3.5/site-packa

Py4JError: An error occurred while calling None.None